Pybacktestchain code

In [184]:
from pybacktestchain.data_module import FirstTwoMoments
from pybacktestchain.broker import Broker, Backtest, StopLoss
from pybacktestchain.blockchain import Blockchain
from pybacktestchain.utils import generate_random_name
from datetime import datetime
import pandas as pd
import logging

# Set verbosity for logging
verbose = False  # Set to True to enable logging, or False to suppress it

# Custom Broker Class
class CustomBroker(Broker):
    def initialize_blockchain(self, name: str):
        """Always create a fresh blockchain."""
        self.blockchain = Blockchain(name)
        self.blockchain.store()  # Overwrite the blockchain file
        if self.verbose:
            logging.info(f"Blockchain with name {name} re-initialized and stored.")

    def reset(self, cash: float):
        """Resets the broker state."""
        self.cash = cash
        self.positions = {}
        self.transaction_log = pd.DataFrame(columns=['Date', 'Action', 'Ticker', 'Quantity', 'Price', 'Cash'])
        self.entry_prices = {}

# Custom Backtest Class
class CustomBacktest(Backtest):
    def __post_init__(self):
        # Generate a random name for the backtest
        self.backtest_name = generate_random_name()

        # Use the custom broker instead of the default one
        self.broker = CustomBroker(cash=self.initial_cash, verbose=self.verbose)
        self.broker.initialize_blockchain(self.name_blockchain)

# Function to Run a Fresh Backtest
def run_clean_backtest():
    # Create and run a new backtest instance
    backtest = CustomBacktest(
        initial_date=datetime(2019, 1, 1),
        final_date=datetime(2020, 1, 1),
        information_class=FirstTwoMoments,
        risk_model=StopLoss,
        name_blockchain='backtest',
        verbose=verbose
    )
    # Run the backtest
    backtest.run_backtest()

# Execute the Backtest
run_clean_backtest()


INFO:root:Running backtest from 2019-01-01 00:00:00 to 2020-01-01 00:00:00.
INFO:root:Retrieving price data for universe
c:\Users\ASUS\anaconda3\Lib\site-packages\pandas\core\frame.py:11211: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
c:\Users\ASUS\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\Users\ASUS\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
INFO:root:-----------------------------------
INFO:root:Rebalancing portfolio at 2019-01-31 00:00:00
c:\Users\ASUS\anaconda3\Lib\site-packages\pybacktestchain\broker.py:115: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old be

code that works to find final pf value

In [202]:
import pandas as pd
import logging
from dataclasses import dataclass
from datetime import datetime, timedelta
import os
from pybacktestchain.data_module import FirstTwoMoments, get_stocks_data
from pybacktestchain.broker import Backtest, StopLoss
import pandas as pd

# Set verbosity for logging
verbose = True  # Set to True to enable logging, or False to suppress it

# Custom Risk Model / Strategy
class CustomStrategy(StopLoss):  # Inherit from StopLoss to provide required methods
    def __init__(self, threshold=1):
        super().__init__(threshold=threshold)  # Initialize StopLoss logic
        self.threshold = threshold

    def decide_trade(self, current_price, historical_data):
        # Example strategy: Moving average-based logic
        if 'Close' in historical_data:
            moving_average = historical_data['Close'].rolling(window=10).mean().iloc[-1]
            if current_price < moving_average:
                return 'buy'
            elif current_price > moving_average:
                return 'sell'
        return 'hold'

    def trigger_stop_loss(self, t, portfolio, prices, broker):
        # Implement stop-loss logic (if required)
        super().trigger_stop_loss(t, portfolio, prices, broker)

def run_backtests_and_get_values():
    # Backtest for original strategy
    backtest_original = Backtest(
        initial_date=datetime(2019, 1, 6),
        final_date=datetime(2020, 1, 1),
        information_class=FirstTwoMoments,
        risk_model=StopLoss,
        name_blockchain='original_strategy_v3',
        verbose=verbose
    )
    backtest_original.run_backtest()

    # Convert initial and final dates to string
    start_date = backtest_original.initial_date.strftime('%Y-%m-%d')
    end_date = backtest_original.final_date.strftime('%Y-%m-%d')

    try:
        # Clean tickers (strip "$" if present in tickers)
        cleaned_tickers = [ticker.lstrip('$') for ticker in backtest.universe]
        final_date_prices = get_stocks_data(cleaned_tickers, start_date, end_date)
        latest_prices = final_date_prices.groupby('ticker')['Adj Close'].last().to_dict()
    except ValueError:
        logging.warning("Using last available transaction prices due to missing final prices.")
        transaction_log = backtest.broker.get_transaction_log()
        latest_prices = transaction_log.groupby('Ticker')['Price'].last().to_dict()

    # Retrieve final portfolio value for original strategy
    original_portfolio_value = backtest.broker.get_portfolio_value(latest_prices)

    # Backtest for custom strategy
    backtest_custom = Backtest(
        initial_date=datetime(2019, 1, 6),
        final_date=datetime(2020, 1, 1),
        information_class=FirstTwoMoments,
        risk_model=CustomStrategy,
        name_blockchain='custom_strategy_v3',
        verbose=verbose
    )
    backtest_custom.run_backtest()

    # Retrieve final portfolio value for custom strategy
    custom_portfolio_value = backtest_custom.broker.get_portfolio_value(latest_prices)

    return original_portfolio_value, custom_portfolio_value

# Main Execution
original_value, custom_value = run_backtests_and_get_values()

# Output final portfolio values
print(f"Final portfolio value (Original Strategy): {original_value:.2f}")
print(f"Final portfolio value (Custom Strategy): {custom_value:.2f}")




INFO:root:Running backtest from 2019-01-06 00:00:00 to 2020-01-01 00:00:00.
INFO:root:Retrieving price data for universe
c:\Users\ASUS\anaconda3\Lib\site-packages\pandas\core\frame.py:11211: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
c:\Users\ASUS\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\Users\ASUS\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
INFO:root:Stop loss triggered for NVDA at 2019-01-08 00:00:00. Selling all shares.
INFO:root:Stop loss triggered for CSCO at 2019-01-08 00:00:00. Selling all shares.
INFO:root:-----------------------------------
INFO:root:Rebalancing portfolio at 2019-01-31 00:00:00
INFO:root:-----------------------------------
INFO:root:Rebalancing portfolio at 2019-02-28 00:00:00
INFO:root:Buying as many shares of

Final portfolio value (Original Strategy): 144950665.57
Final portfolio value (Custom Strategy): 151799767.48


CLASS TO ANALYZE PERF OF A BACKTEST

In [244]:
from pybacktestchain.broker import Backtest

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

class PerformanceAnalyzer:
    def __init__(self, transaction_log: pd.DataFrame, initial_cash: float):
        self.transaction_log = transaction_log
        self.initial_cash = initial_cash
        self.portfolio_value = self.calculate_portfolio_value()

    def calculate_portfolio_value(self):
        """Calculate the portfolio value over time."""
        self.transaction_log['Cumulative Cash'] = self.transaction_log['Cash'].cumsum()
        return self.transaction_log[['Date', 'Cumulative Cash']]

    def calculate_cagr(self):
        """Calculate the Compound Annual Growth Rate (CAGR)."""
        start_value = self.initial_cash
        end_value = self.portfolio_value['Cumulative Cash'].iloc[-1]
        n_years = (self.transaction_log['Date'].iloc[-1] - self.transaction_log['Date'].iloc[0]).days / 365.25
        return (end_value / start_value) ** (1 / n_years) - 1

    def calculate_sharpe_ratio(self, risk_free_rate=0.02):
        """Calculate the Sharpe Ratio."""
        returns = self.transaction_log['Cumulative Cash'].pct_change().dropna()
        excess_returns = returns - risk_free_rate / 252
        return excess_returns.mean() / excess_returns.std() * np.sqrt(252)

    def calculate_max_drawdown(self):
        """Calculate the maximum drawdown."""
        cumulative = self.transaction_log['Cumulative Cash'].cummax()
        drawdowns = self.transaction_log['Cumulative Cash'] / cumulative - 1
        return drawdowns.min()

    def calculate_win_rate(self):
        """Calculate the win rate of trades."""
        wins = self.transaction_log[self.transaction_log['Action'] == 'SELL']['Price'].pct_change().dropna()
        return (wins > 0).sum() / len(wins)

    def generate_report(self):
        """Generate a performance report."""
        metrics = {
            "CAGR": self.calculate_cagr(),
            "Sharpe Ratio": self.calculate_sharpe_ratio(),
            "Max Drawdown": self.calculate_max_drawdown(),
            "Win Rate": self.calculate_win_rate(),
        }
        return metrics



CLASS TO STOCK ALL DIFFERENT STRATEGIES

In [245]:
from pybacktestchain.broker import StopLoss
import logging
import random

class TrailingStop(StopLoss):
    """A trailing stop risk model."""
    def __init__(self, trailing_threshold=0.1):
        super().__init__()
        self.trailing_threshold = trailing_threshold
        self.high_watermarks = {}  # Track highest prices for each asset

    def trigger_stop_loss(self, t, portfolio, prices, broker):
        """
        Trigger a trailing stop-loss if the price falls below the high watermark
        by more than the threshold.
        """
        for ticker, position in broker.positions.items():
            current_price = prices.get(ticker)
            if current_price is None:
                continue

            # Update the high watermark
            if ticker not in self.high_watermarks:
                self.high_watermarks[ticker] = current_price
            self.high_watermarks[ticker] = max(self.high_watermarks[ticker], current_price)

            # Check for trailing stop
            loss_threshold = self.high_watermarks[ticker] * (1 - self.trailing_threshold)
            if current_price < loss_threshold:
                broker.sell(ticker, position.quantity, current_price, t)
                logging.info(f"Trailing stop triggered for {ticker} at {t}. Sold {position.quantity} shares.")

class RiskParityModel(StopLoss):
    """A risk parity model that allocates weights to minimize portfolio volatility."""
    def trigger_stop_loss(self, t, portfolio, prices, broker):
        """
        Adjust the portfolio to balance risk based on volatility estimates.
        """
        # Fetch historical data
        volatilities = {}
        for ticker, position in broker.positions.items():
            # Fetch historical prices (replace `fetch_historical_prices` with your actual function)
            historical_prices = self.fetch_historical_prices(ticker, t)

            if isinstance(historical_prices, pd.Series):
                volatilities[ticker] = historical_prices.pct_change().std()
            else:
                logging.warning(f"Could not compute volatility for {ticker}: Missing historical prices.")

        # Compute weights inversely proportional to volatility
        total_risk = sum(1 / v for v in volatilities.values() if v > 0)
        weights = {ticker: (1 / volatilities[ticker]) / total_risk for ticker in volatilities if volatilities[ticker] > 0}

        # Rebalance the portfolio based on the computed weights
        total_value = broker.get_portfolio_value(prices)
        for ticker, weight in weights.items():
            target_value = total_value * weight
            current_value = broker.positions.get(ticker, 0).quantity * prices.get(ticker, 0)
            diff_value = target_value - current_value
            quantity_to_trade = int(diff_value / prices.get(ticker, 1))

            if quantity_to_trade > 0:
                broker.buy(ticker, quantity_to_trade, prices[ticker], t)
            elif quantity_to_trade < 0:
                broker.sell(ticker, abs(quantity_to_trade), prices[ticker], t)

    def fetch_historical_prices(self, ticker, t):
        """
        Fetch historical price data for a given ticker and time (t).
        Replace this with your actual implementation for fetching historical prices.
        """
        from pybacktestchain.data_module import get_stocks_data
        # Assume a 1-year lookback window
        start_date = (t - timedelta(days=365)).strftime('%Y-%m-%d')
        end_date = t.strftime('%Y-%m-%d')

        try:
            data = get_stocks_data([ticker], start_date, end_date)
            return data[data['ticker'] == ticker]['Adj Close']
        except Exception as e:
            logging.error(f"Error fetching historical prices for {ticker}: {e}")
            return pd.Series()  # Return an empty Series if fetching fails
        


class RandomStrategy(StopLoss):
    def __init__(self, threshold=0.1, buy_probability=0.3, sell_probability=0.3, max_trade_fraction=0.1):
        super().__init__(threshold=threshold)
        self.buy_probability = buy_probability
        self.sell_probability = sell_probability
        self.max_trade_fraction = max_trade_fraction

    def trigger_stop_loss(self, t, portfolio, prices, broker):
        import random

        for ticker, position in list(broker.positions.items()):
            current_price = prices.get(ticker)
            if current_price is None:
                logging.warning(f"Price for {ticker} not available. Skipping.")
                continue

            action = random.choices(
                ["buy", "sell", "hold"],
                weights=[self.buy_probability, self.sell_probability, 1 - (self.buy_probability + self.sell_probability)],
                k=1
            )[0]

            if action == "buy":
                cash_available = broker.get_cash_balance()
                max_allocation = self.max_trade_fraction * cash_available
                quantity_to_buy = int(max_allocation / current_price)
                if quantity_to_buy > 0:
                    broker.buy(ticker, quantity_to_buy, current_price, t)

            elif action == "sell" and position.quantity > 0:
                broker.sell(ticker, position.quantity, current_price, t)






In [246]:
def run_and_analyze_backtest():
    # Run a backtest
    backtest = CustomBacktest(
        initial_date=datetime(2019, 1, 1),
        final_date=datetime(2020, 1, 1),
        information_class=FirstTwoMoments,
        risk_model=RiskParityModel,
        name_blockchain='test_backtest',
        verbose=True
    )
    backtest.run_backtest()

    # Extract transaction log
    transaction_log = backtest.broker.get_transaction_log()

    # Analyze performance
    analyzer = PerformanceAnalyzer(transaction_log=transaction_log, initial_cash=1000000)
    metrics = analyzer.generate_report()
    
    # Print metrics
    print("Performance Metrics:")
    for key, value in metrics.items():
        print(f"{key}: {value:.2%}")



# Main Execution
if __name__ == "__main__":
    run_and_analyze_backtest()

INFO:root:Blockchain with name test_backtest re-initialized and stored.
INFO:root:Running backtest from 2019-01-01 00:00:00 to 2020-01-01 00:00:00.
INFO:root:Retrieving price data for universe
c:\Users\ASUS\anaconda3\Lib\site-packages\pandas\core\frame.py:11211: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
c:\Users\ASUS\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\Users\ASUS\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
INFO:root:-----------------------------------
INFO:root:Rebalancing portfolio at 2019-01-31 00:00:00
c:\Users\ASUS\anaconda3\Lib\site-packages\pybacktestchain\broker.py:115: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or

Performance Metrics:
CAGR: 2439.78%
Sharpe Ratio: 78.04%
Max Drawdown: 0.00%
Win Rate: 50.55%


grading system for backtest

In [248]:
def assign_grade(metrics):
    """
    Assign a grade based on performance metrics.
    :param metrics: Dictionary with performance metrics (CAGR, Sharpe Ratio, Max Drawdown, Win Rate).
    :return: Tuple (grade, tokens).
    """
    if metrics["CAGR"] > 0.30 and metrics["Sharpe Ratio"] > 1.5 and metrics["Max Drawdown"] > -0.2:
        grade = "A"
        tokens = 100  # High performers
    elif metrics["CAGR"] > 0.20 and metrics["Sharpe Ratio"] > 1.0 and metrics["Max Drawdown"] > -0.3:
        grade = "B"
        tokens = 50  # Moderate performers
    elif metrics["CAGR"] > 0.10 and metrics["Sharpe Ratio"] > 0.5 and metrics["Max Drawdown"] > -0.4:
        grade = "C"
        tokens = 20  # Low performers
    else:
        grade = "D"
        tokens = 5  # Minimal reward
    return grade, tokens

def evaluate_submissions(submissions):
    """
    Evaluate user submissions and assign grades and tokens.
    :param submissions: List of user submissions with metrics.
    :return: List of evaluations with grades and tokens.
    """
    evaluations = []
    for submission in submissions:
        metrics = submission["metrics"]
        grade, tokens = assign_grade(metrics)
        evaluations.append({
            "user_address": submission["user_address"],
            "strategy_name": submission["strategy_name"],
            "metrics": metrics,
            "grade": grade,
            "tokens": tokens,
        })
    return evaluations

###EXAMPLE

# Example submissions with metrics
submissions = [
    {"strategy_name": "Strategy A", "user_address": "0xUser1Address", "metrics": {"CAGR": 0.35, "Sharpe Ratio": 1.8, "Max Drawdown": -0.15, "Win Rate": 0.6}},
    {"strategy_name": "Strategy B", "user_address": "0xUser2Address", "metrics": {"CAGR": 0.25, "Sharpe Ratio": 1.2, "Max Drawdown": -0.25, "Win Rate": 0.55}},
    {"strategy_name": "Strategy C", "user_address": "0xUser3Address", "metrics": {"CAGR": 0.15, "Sharpe Ratio": 0.8, "Max Drawdown": -0.35, "Win Rate": 0.5}},
    {"strategy_name": "Strategy D", "user_address": "0xUser4Address", "metrics": {"CAGR": 0.05, "Sharpe Ratio": 0.3, "Max Drawdown": -0.5, "Win Rate": 0.4}},
]

# Evaluate submissions
evaluations = evaluate_submissions(submissions)

# Display results
for evaluation in evaluations:
    print(f"User {evaluation['user_address']} (Strategy: {evaluation['strategy_name']}):")
    print(f"  Grade: {evaluation['grade']}, Tokens: {evaluation['tokens']}")
    print(f"  Metrics: {evaluation['metrics']}")


User 0xUser1Address (Strategy: Strategy A):
  Grade: A, Tokens: 100
  Metrics: {'CAGR': 0.35, 'Sharpe Ratio': 1.8, 'Max Drawdown': -0.15, 'Win Rate': 0.6}
User 0xUser2Address (Strategy: Strategy B):
  Grade: B, Tokens: 50
  Metrics: {'CAGR': 0.25, 'Sharpe Ratio': 1.2, 'Max Drawdown': -0.25, 'Win Rate': 0.55}
User 0xUser3Address (Strategy: Strategy C):
  Grade: C, Tokens: 20
  Metrics: {'CAGR': 0.15, 'Sharpe Ratio': 0.8, 'Max Drawdown': -0.35, 'Win Rate': 0.5}
User 0xUser4Address (Strategy: Strategy D):
  Grade: D, Tokens: 5
  Metrics: {'CAGR': 0.05, 'Sharpe Ratio': 0.3, 'Max Drawdown': -0.5, 'Win Rate': 0.4}


#TO SEE IF NEEDED, OTHERWISE DELETE

2 strategies where 1 custom one with a moving average dynamic

In [188]:
import pandas as pd
import logging
from dataclasses import dataclass
from datetime import datetime, timedelta
import os
from pybacktestchain.data_module import FirstTwoMoments, get_stocks_data
from pybacktestchain.broker import Backtest, StopLoss
import pandas as pd

# Set verbosity for logging
verbose = True  # Set to True to enable logging, or False to suppress it

# Custom Risk Model / Strategy
class CustomStrategy(StopLoss):  # Inherit from StopLoss to provide required methods
    def __init__(self, threshold=1):
        super().__init__(threshold=threshold)  # Initialize StopLoss logic
        self.threshold = threshold

    def decide_trade(self, current_price, historical_data):
        # Example strategy: Moving average-based logic
        if 'Close' in historical_data:
            moving_average = historical_data['Close'].rolling(window=10).mean().iloc[-1]
            if current_price < moving_average:
                return 'buy'
            elif current_price > moving_average:
                return 'sell'
        return 'hold'

    def trigger_stop_loss(self, t, portfolio, prices, broker):
        # Implement stop-loss logic (if required)
        super().trigger_stop_loss(t, portfolio, prices, broker)

def run_backtests_and_get_values():
    # Backtest for original strategy
    backtest_original = Backtest(
        initial_date=datetime(2019, 1, 6),
        final_date=datetime(2020, 1, 1),
        information_class=FirstTwoMoments,
        risk_model=StopLoss,
        name_blockchain='original_strategy_v3',
        verbose=verbose
    )
    backtest_original.run_backtest()

    # Convert initial and final dates to string
    start_date = backtest_original.initial_date.strftime('%Y-%m-%d')
    end_date = backtest_original.final_date.strftime('%Y-%m-%d')

    try:
        # Clean tickers (strip "$" if present in tickers)
        cleaned_tickers = [ticker.lstrip('$') for ticker in backtest.universe]
        final_date_prices = get_stocks_data(cleaned_tickers, start_date, end_date)
        latest_prices = final_date_prices.groupby('ticker')['Adj Close'].last().to_dict()
    except ValueError:
        logging.warning("Using last available transaction prices due to missing final prices.")
        transaction_log = backtest.broker.get_transaction_log()
        latest_prices = transaction_log.groupby('Ticker')['Price'].last().to_dict()

    # Retrieve final portfolio value for original strategy
    original_portfolio_value = backtest.broker.get_portfolio_value(latest_prices)

    # Backtest for custom strategy
    backtest_custom = Backtest(
        initial_date=datetime(2019, 1, 6),
        final_date=datetime(2020, 1, 1),
        information_class=FirstTwoMoments,
        risk_model=CustomStrategy,
        name_blockchain='custom_strategy_v3',
        verbose=verbose
    )
    backtest_custom.run_backtest()

    # Retrieve final portfolio value for custom strategy
    custom_portfolio_value = backtest_custom.broker.get_portfolio_value(latest_prices)

    return original_portfolio_value, custom_portfolio_value

# Main Execution
original_value, custom_value = run_backtests_and_get_values()

# Output final portfolio values
print(f"Final portfolio value (Original Strategy): {original_value:.2f}")
print(f"Final portfolio value (Custom Strategy): {custom_value:.2f}")




INFO:root:Running backtest from 2019-01-06 00:00:00 to 2020-01-01 00:00:00.
INFO:root:Retrieving price data for universe
c:\Users\ASUS\anaconda3\Lib\site-packages\pandas\core\frame.py:11211: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
c:\Users\ASUS\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\Users\ASUS\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
INFO:root:Stop loss triggered for NVDA at 2019-01-08 00:00:00. Selling all shares.
INFO:root:Stop loss triggered for CSCO at 2019-01-08 00:00:00. Selling all shares.
INFO:root:-----------------------------------
INFO:root:Rebalancing portfolio at 2019-01-31 00:00:00
INFO:root:-----------------------------------
INFO:root:Rebalancing portfolio at 2019-02-28 00:00:00
INFO:root:Buying as many shares of

Final portfolio value (Original Strategy): 129521466.67
Final portfolio value (Custom Strategy): 135641518.63


In [192]:
import pandas as pd
import logging
from dataclasses import dataclass
from datetime import datetime, timedelta
import os
from pybacktestchain.data_module import FirstTwoMoments, get_stocks_data
from pybacktestchain.broker import Broker, Backtest, StopLoss
from pybacktestchain.blockchain import Blockchain
from pybacktestchain.utils import generate_random_name

# Set verbosity for logging
verbose = True  # Set to True to enable logging, or False to suppress it

# Custom Broker Class
class CustomBroker(Broker):
    def initialize_blockchain(self, name: str):
        """Always create a fresh blockchain."""
        self.blockchain = Blockchain(name)
        self.blockchain.store()  # Overwrite the blockchain file
        if self.verbose:
            logging.info(f"Blockchain with name {name} re-initialized and stored.")

    def reset(self, cash: float):
        """Resets the broker state."""
        self.cash = cash
        self.positions = {}
        self.transaction_log = pd.DataFrame(columns=['Date', 'Action', 'Ticker', 'Quantity', 'Price', 'Cash'])
        self.entry_prices = {}

# Custom Backtest Class
class CustomBacktest(Backtest):
    def __post_init__(self):
        # Generate a random name for the backtest
        self.backtest_name = generate_random_name()

        # Use the custom broker instead of the default one
        self.broker = CustomBroker(cash=self.initial_cash, verbose=self.verbose)
        self.broker.initialize_blockchain(self.name_blockchain)

# Custom Risk Model / Strategy
class CustomStrategy(StopLoss):
    def __init__(self, threshold=1):
        super().__init__(threshold=threshold)  # Initialize StopLoss logic
        self.threshold = threshold

    def decide_trade(self, current_price, historical_data):
        # Example strategy: Moving average-based logic
        if 'Close' in historical_data:
            moving_average = historical_data['Close'].rolling(window=10).mean().iloc[-1]
            if current_price < moving_average:
                return 'buy'
            elif current_price > moving_average:
                return 'sell'
        return 'hold'

    def trigger_stop_loss(self, t, portfolio, prices, broker):
        # Implement stop-loss logic (if required)
        super().trigger_stop_loss(t, portfolio, prices, broker)

# Function to Run Backtests and Get Portfolio Values
def run_backtests_and_get_values():
    # Backtest for original strategy
    backtest_original = CustomBacktest(
        initial_date=datetime(2019, 1, 6),
        final_date=datetime(2020, 1, 1),
        information_class=FirstTwoMoments,
        risk_model=StopLoss,
        name_blockchain='original_strategy_v3',
        verbose=verbose
    )
    backtest_original.run_backtest()

    # Convert initial and final dates to string
    start_date = backtest_original.initial_date.strftime('%Y-%m-%d')
    end_date = backtest_original.final_date.strftime('%Y-%m-%d')

    try:
        # Clean tickers (strip "$" if present in tickers)
        cleaned_tickers = [ticker.lstrip('$') for ticker in backtest_original.universe]
        final_date_prices = get_stocks_data(cleaned_tickers, start_date, end_date)
        latest_prices = final_date_prices.groupby('ticker')['Adj Close'].last().to_dict()
    except ValueError:
        logging.warning("Using last available transaction prices due to missing final prices.")
        transaction_log = backtest_original.broker.get_transaction_log()
        latest_prices = transaction_log.groupby('Ticker')['Price'].last().to_dict()

    # Retrieve final portfolio value for original strategy
    original_portfolio_value = backtest_original.broker.get_portfolio_value(latest_prices)

    # Backtest for custom strategy
    backtest_custom = CustomBacktest(
        initial_date=datetime(2019, 1, 6),
        final_date=datetime(2020, 1, 1),
        information_class=FirstTwoMoments,
        risk_model=CustomStrategy,
        name_blockchain='custom_strategy_v3',
        verbose=verbose
    )
    backtest_custom.run_backtest()

    # Retrieve final portfolio value for custom strategy
    custom_portfolio_value = backtest_custom.broker.get_portfolio_value(latest_prices)

    return original_portfolio_value, custom_portfolio_value

# Main Execution
if __name__ == "__main__":
    original_value, custom_value = run_backtests_and_get_values()

    # Output final portfolio values
    print(f"Final portfolio value (Original Strategy): {original_value:.2f}")
    print(f"Final portfolio value (Custom Strategy): {custom_value:.2f}")


INFO:root:Blockchain with name original_strategy_v3 re-initialized and stored.
INFO:root:Running backtest from 2019-01-06 00:00:00 to 2020-01-01 00:00:00.
INFO:root:Retrieving price data for universe
c:\Users\ASUS\anaconda3\Lib\site-packages\pandas\core\frame.py:11211: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
c:\Users\ASUS\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\Users\ASUS\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
INFO:root:-----------------------------------
INFO:root:Rebalancing portfolio at 2019-01-31 00:00:00
c:\Users\ASUS\anaconda3\Lib\site-packages\pybacktestchain\broker.py:115: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude e

Final portfolio value (Original Strategy): 1688382.38
Final portfolio value (Custom Strategy): 1688382.38


adding a strategy : mean reversion

In [149]:
from pybacktestchain.broker import Broker, StopLoss, Backtest
from pybacktestchain.data_module import FirstTwoMoments
from datetime import datetime
from dataclasses import dataclass
import pandas as pd


@dataclass
class AggressiveMeanReversionStrategy(StopLoss):
    def __init__(self, threshold=0.1):
        super().__init__(threshold=threshold)  # Retain stop-loss logic
        self.actions_log = []  # Log actions for analysis

    def decide_trade(self, current_price, historical_data):
        """Decides whether to buy, sell, or hold based on mean-reversion logic."""
        action = 'hold'  # Default action

        # Ensure historical data has sufficient data points for SMA calculation
        if len(historical_data) >= 20:  # Example: 20-day SMA
            sma = historical_data['Close'].rolling(window=20).mean().iloc[-1]
            deviation = (current_price - sma) / sma  # Deviation from mean

            # Aggressive buy if price is significantly below the SMA
            if deviation <= -0.1:  # 10% below SMA
                action = 'buy'
            # Aggressive sell if price is significantly above the SMA
            elif deviation >= 0.1:  # 10% above SMA
                action = 'sell'

        # Log the decision
        self.actions_log.append({
            'Date': historical_data.index[-1],
            'Price': current_price,
            'Action': action,
            'SMA': sma if len(historical_data) >= 20 else None,
            'Deviation': deviation if len(historical_data) >= 20 else None
        })

        return action

    def trigger_stop_loss(self, t, portfolio, prices, broker):
        """Applies stop-loss and evaluates mean-reversion trades."""
        # Apply the inherited stop-loss logic
        super().trigger_stop_loss(t, portfolio, prices, broker)

        # Simulate historical price data for each ticker
        # Replace with actual historical data in real use
        historical_data = {
            ticker: pd.DataFrame({
                'Close': [prices[ticker] * (1 + 0.01 * i) for i in range(-20, 0)]  # Mock rolling prices
            }, index=pd.date_range(start=t - pd.Timedelta(days=19), end=t))
            for ticker in portfolio.keys()
        }

        # Apply mean-reversion logic
        for ticker, position in broker.positions.items():
            if ticker in prices:
                historical = historical_data.get(ticker, pd.DataFrame())
                action = self.decide_trade(prices[ticker], historical)
                if action == 'buy':
                    broker.buy(ticker, 1, prices[ticker], t)  # Adjust quantity as needed
                elif action == 'sell':
                    broker.sell(ticker, 1, prices[ticker], t)

    def get_actions_log(self):
        """Returns the log of all trade actions."""
        return pd.DataFrame(self.actions_log)


# Backtest for Aggressive Mean Reversion Strategy
backtest = Backtest(
    initial_date=datetime(2019, 1, 1),
    final_date=datetime(2020, 1, 1),
    information_class=FirstTwoMoments,
    risk_model=AggressiveMeanReversionStrategy,  # Use the new strategy
    name_blockchain='mean_reversion_strategy',
    verbose=True
)
backtest.run_backtest()

# Retrieve the final portfolio value
market_prices = backtest.broker.get_transaction_log().iloc[-1]['Price']  # Simplified example for last price
final_portfolio_value = backtest.broker.get_portfolio_value({ticker: market_prices for ticker in backtest.universe})
print(f"Final Portfolio Value: ${final_portfolio_value:.2f}")


INFO:root:Running backtest from 2019-01-01 00:00:00 to 2020-01-01 00:00:00.
INFO:root:Retrieving price data for universe
c:\Users\ASUS\anaconda3\Lib\site-packages\pandas\core\frame.py:11211: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
c:\Users\ASUS\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\Users\ASUS\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
INFO:root:Stop loss triggered for NVDA at 2019-01-03 00:00:00. Selling all shares.
INFO:root:Stop loss triggered for NFLX at 2019-01-03 00:00:00. Selling all shares.
INFO:root:Stop loss triggered for CSCO at 2019-01-03 00:00:00. Selling all shares.
INFO:root:-----------------------------------
INFO:root:Rebalancing portfolio at 2019-01-31 00:00:00
INFO:root:-----------------------------------
INFO:r

RuntimeError: dictionary changed size during iteration

First, we create a code that leverages pybacktestchain to run a backtest on a user-defined trading strategy, evaluating its profitability and innovativeness. 
It calculates key performance metrics like net profit, Sharpe ratio, and maximum drawdown, while comparing the strategy to historical ones using pattern correlation. 
Results are stored in a blockchain to ensure transparency and traceability.

In [2]:
from pybacktestchain.data_module import FirstTwoMoments
from pybacktestchain.broker import Backtest, StopLoss
from pybacktestchain.blockchain import load_blockchain
from datetime import datetime
import pandas as pd
import numpy as np
from io import StringIO
import pickle

# Ensures that the user's strategy is correctly applied to the market data.
def apply_user_strategy(strategy_function, data):
    """
    Apply the user's strategy to the data.
    :param strategy_function: User-defined strategy function.
    :param data: Historical market data (DataFrame).
    :return: DataFrame with added 'Signal' column.
    """
    try:
        # Check if the strategy function can be applied
        data = strategy_function(data)
        if 'Signal' not in data.columns:
            raise ValueError("The strategy function must add a 'Signal' column to the data.")
        return data
    except Exception as e:
        print(f"Error applying user strategy: {e}")
        raise

# Initialize the backtest
backtest = Backtest(
    initial_date=datetime(2019, 1, 1),
    final_date=datetime(2020, 1, 1),
    information_class=FirstTwoMoments,
    risk_model=StopLoss,
    name_blockchain='user_strategy_analysis',
    verbose=True
)

# Example of user-defined strategy (replace with user input)
def user_defined_strategy(data):
    """
    Example: Moving Average Crossover Strategy.
    Users can replace this with their own strategy.
    """
    short_ma = data['Close'].rolling(window=10).mean()
    long_ma = data['Close'].rolling(window=50).mean()
    data['Signal'] = np.where(short_ma > long_ma, 1, -1)  # 1: Buy, -1: Sell
    return data

# Apply the user's strategy to the backtest
try:
    backtest.strategy = lambda data: apply_user_strategy(user_defined_strategy, data)
    backtest.run_backtest()
except Exception as e:
    print(f"Failed to run the backtest: {e}")

# Load the blockchain
blockchain = load_blockchain('user_strategy_analysis')

# Extract trades from a single block
def extract_trades_from_block(block):
    """
    Extract trades from a block's data field.
    """
    if block.data:
        try:
            # Parse the data string into a DataFrame
            df = pd.read_csv(StringIO(block.data), delim_whitespace=True)
            return df
        except Exception as e:
            print(f"Error parsing block data: {e}")
            return None
    return None

# Extract all trades from the blockchain
def extract_all_trades(blockchain):
    """
    Extract trades from all blocks in the blockchain.
    """
    trades = []
    for block in blockchain.chain:
        trades_df = extract_trades_from_block(block)
        if trades_df is not None:
            trades.append(trades_df)
    # Combine all trade data into a single DataFrame
    if trades:
        return pd.concat(trades, ignore_index=True)
    return pd.DataFrame()

# Analyze profitability
def calculate_profitability(trades_df):
    """
    Calculate net profit and other key performance metrics.
    """
    if trades_df.empty:
        print("No trades found.")
        return {"Net Profit": 0, "Sharpe Ratio": 0}

    # Calculate net profit
    trades_df['Profit'] = trades_df['Quantity'] * trades_df['Price']
    net_profit = trades_df['Profit'].sum()

    # Example: Calculate Sharpe Ratio
    sharpe_ratio = trades_df['Profit'].mean() / trades_df['Profit'].std() if trades_df['Profit'].std() > 0 else 0

    print(f"Net Profit: {net_profit}")
    print(f"Sharpe Ratio: {sharpe_ratio}")
    return {"Net Profit": net_profit, "Sharpe Ratio": sharpe_ratio}

# Analyze innovation of the strategy
def analyze_innovation(trades_df):
    """
    Analyze the innovation of the user's strategy.
    Compare the strategy's signals or trading patterns against historical strategies.
    """
    if trades_df.empty:
        print("No trades found to analyze innovation.")
        return {"Innovation Score": 0}

    # Example: Compare the user's trades to a dummy historical pattern
    # (In a real-world scenario, you would load historical strategies from a database)
    historical_signals = np.random.choice([-1, 1], size=len(trades_df))  # Dummy historical signals
    user_signals = trades_df['Signal'] if 'Signal' in trades_df else []

    # Calculate innovation as 1 - correlation with historical signals
    correlation = np.corrcoef(user_signals, historical_signals)[0, 1] if len(user_signals) > 1 else 0
    innovation_score = 1 - correlation

    print(f"Innovation Score: {innovation_score}")
    return {"Innovation Score": innovation_score}

# Save blockchain to a file using pickle
def save_blockchain(blockchain, filename='blockchain.pkl'):
    """Save the blockchain object to a file."""
    with open(filename, 'wb') as file:
        pickle.dump(blockchain, file)
    print(f"Blockchain saved to {filename}")

# Extract and analyze trades
trades_df = extract_all_trades(blockchain)
print(trades_df.head())  # Display the first few trades

# Calculate profitability metrics
profitability_metrics = calculate_profitability(trades_df)
print(profitability_metrics)

# Analyze innovation
innovation_metrics = analyze_innovation(trades_df)
print(innovation_metrics)

# Save the blockchain
save_blockchain(blockchain)


INFO:root:Running backtest from 2019-01-01 00:00:00 to 2020-01-01 00:00:00.
INFO:root:Retrieving price data for universe
c:\Users\ASUS\anaconda3\Lib\site-packages\pandas\core\frame.py:11211: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
c:\Users\ASUS\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\Users\ASUS\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
INFO:root:Stop loss triggered for NVDA at 2019-01-03 00:00:00. Selling all shares.
INFO:root:Stop loss triggered for NFLX at 2019-01-03 00:00:00. Selling all shares.
INFO:root:Stop loss triggered for CSCO at 2019-01-03 00:00:00. Selling all shares.
INFO:root:-----------------------------------
INFO:root:Rebalancing portfolio at 2019-01-31 00:00:00
INFO:root:-----------------------------------
INFO:r

         Date Action Ticker  Quantity      Price          Cash
0  2019-01-31    BUY   NVDA    290529   3.407105  10137.200000
1  2019-01-31    BUY   TSLA       492  20.584667      9.544189
2  2019-02-28   SELL   NVDA     12749   3.853979  49143.920000
3  2019-02-28    BUY   TSLA      2342  20.982668      2.515596
4  2019-03-06   SELL   TSLA      2834  18.436001  52250.140000
Net Profit: 1361303086.610451
Sharpe Ratio: 0.4910392052872515
{'Net Profit': 1361303086.610451, 'Sharpe Ratio': 0.4910392052872515}
Innovation Score: 1
{'Innovation Score': 1}
Blockchain saved to blockchain.pkl


C:\Users\ASUS\AppData\Local\Temp\ipykernel_20080\1436809057.py:67: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(block.data), delim_whitespace=True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_20080\1436809057.py:67: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(block.data), delim_whitespace=True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_20080\1436809057.py:67: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(block.data), delim_whitespace=True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_20080\1436809057.py:67: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version

In [44]:
from pybacktestchain.data_module import FirstTwoMoments
from pybacktestchain.broker import Backtest, StopLoss
from pybacktestchain.blockchain import load_blockchain
from datetime import datetime
import pandas as pd
import numpy as np
from io import StringIO
import pickle

# Ensures that the user's strategy is correctly applied to the market data.
def apply_user_strategy(strategy_function, data):
    try:
        data = strategy_function(data)
        if 'Signal' not in data.columns:
            raise ValueError("The strategy function must add a 'Signal' column to the data.")
        return data
    except Exception as e:
        print(f"Error applying user strategy: {e}")
        raise

# Define historical strategies
def create_historical_strategies():
    """
    Create a list of basic historical strategies.
    Each strategy function operates on the market data and generates signals.
    :return: List of strategy functions.
    """
    strategies = []

    # Strategy 1: Always buy
    def strategy_buy(data):
        data['Signal'] = 1  # Always buy
        return data
    strategies.append(strategy_buy)

    # Strategy 2: Always sell
    def strategy_sell(data):
        data['Signal'] = -1  # Always sell
        return data
    strategies.append(strategy_sell)

    # Strategy 3: Random signals
    def strategy_random(data):
        data['Signal'] = np.random.choice([-1, 1], size=len(data))
        return data
    strategies.append(strategy_random)

    # Strategy 4: Short-term momentum
    def strategy_short_momentum(data):
        short_term_diff = data['Close'].diff()
        data['Signal'] = np.where(short_term_diff > 0, 1, -1)
        return data
    strategies.append(strategy_short_momentum)

    # Strategy 5: Long-term momentum
    def strategy_long_momentum(data):
        long_term_mean = data['Close'].rolling(window=20).mean().diff()
        data['Signal'] = np.where(long_term_mean > 0, 1, -1)
        return data
    strategies.append(strategy_long_momentum)

    # Strategy 6: Mean reversion
    def strategy_mean_reversion(data):
        rolling_mean = data['Close'].rolling(window=10).mean()
        data['Signal'] = np.where(data['Close'] < rolling_mean, 1, -1)
        return data
    strategies.append(strategy_mean_reversion)

    # Strategy 7: Moving average crossover
    def strategy_moving_average(data):
        short_ma = data['Close'].rolling(window=5).mean()
        long_ma = data['Close'].rolling(window=15).mean()
        data['Signal'] = np.where(short_ma > long_ma, 1, -1)
        return data
    strategies.append(strategy_moving_average)

    # Strategy 8: Price above moving average
    def strategy_above_ma(data):
        rolling_mean = data['Close'].rolling(window=10).mean()
        data['Signal'] = np.where(data['Close'] > rolling_mean, 1, -1)
        return data
    strategies.append(strategy_above_ma)

    # Strategy 9: Volatility breakout
    def strategy_volatility_breakout(data):
        rolling_mean = data['Close'].rolling(window=10).mean()
        rolling_std = data['Close'].rolling(window=10).std()
        data['Signal'] = np.where(data['Close'] > rolling_mean + 2 * rolling_std, 1, -1)
        return data
    strategies.append(strategy_volatility_breakout)

    # Strategy 10: RSI-based strategy
    def strategy_rsi(data):
        delta = data['Close'].diff()
        gain = np.where(delta > 0, delta, 0)
        loss = np.where(delta < 0, -delta, 0)
        avg_gain = pd.Series(gain).rolling(window=14).mean()
        avg_loss = pd.Series(loss).rolling(window=14).mean()
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
        data['Signal'] = np.where(rsi < 30, 1, np.where(rsi > 70, -1, 0))
        return data
    strategies.append(strategy_rsi)

    return strategies

historical_strategies = create_historical_strategies()

# Analyze innovation of the strategy
def analyze_innovation(trades_df, historical_strategies):
    if trades_df.empty:
        print("No trades found to analyze innovation.")
        return {"Innovation Score": 0}

    user_signals = trades_df['Signal'] if 'Signal' in trades_df.columns else []

    historical_signals_list = []
    for strategy in historical_strategies:
        try:
            historical_data = trades_df.copy()
            historical_data = strategy(historical_data)
            if 'Signal' in historical_data.columns:
                historical_signals_list.append(historical_data['Signal'].to_numpy())
        except Exception as e:
            print(f"Error applying historical strategy: {e}")

    correlations = []
    for historical_signals in historical_signals_list:
        if len(user_signals) == len(historical_signals):
            correlation = np.corrcoef(user_signals, historical_signals)[0, 1]
            correlations.append(correlation)

    avg_correlation = np.mean(correlations) if correlations else 0
    innovation_score = 1 - avg_correlation

    print(f"Innovation Score: {innovation_score}")
    return {"Innovation Score": innovation_score}

# Initialize the backtest
backtest = Backtest(
    initial_date=datetime(2019, 1, 1),
    final_date=datetime(2020, 1, 1),
    information_class=FirstTwoMoments,
    risk_model=StopLoss,
    name_blockchain='user_strategy_analysis',
    verbose=True
)

# Example of user-defined strategy
def user_defined_strategy(data):
    short_ma = data['Close'].rolling(window=10).mean()
    long_ma = data['Close'].rolling(window=50).mean()
    data['Signal'] = np.where(short_ma > long_ma, 1, -1)
    return data

try:
    backtest.strategy = lambda data: apply_user_strategy(user_defined_strategy, data)
    backtest.run_backtest()
except Exception as e:
    print(f"Failed to run the backtest: {e}")

blockchain = load_blockchain('user_strategy_analysis')

# Extract trades and analyze innovation
def extract_all_trades(blockchain):
    trades = []
    for block in blockchain.chain:
        if block.data:
            try:
                trades_df = pd.read_csv(StringIO(block.data), delim_whitespace=True)
                trades.append(trades_df)
            except Exception as e:
                print(f"Error parsing block data: {e}")
    return pd.concat(trades, ignore_index=True) if trades else pd.DataFrame()

trades_df = extract_all_trades(blockchain)

# Placeholder for calculate_profitability function
def calculate_profitability(trades_df):
    if trades_df.empty:
        print("No trades found.")
        return {"Net Profit": 0, "Sharpe Ratio": 0}

    trades_df['Profit'] = trades_df['Quantity'] * trades_df['Price']
    net_profit = trades_df['Profit'].sum()
    sharpe_ratio = trades_df['Profit'].mean() / trades_df['Profit'].std() if trades_df['Profit'].std() > 0 else 0

    print(f"Net Profit: {net_profit}")
    print(f"Sharpe Ratio: {sharpe_ratio}")
    return {"Net Profit": net_profit, "Sharpe Ratio": sharpe_ratio}

profitability_metrics = calculate_profitability(trades_df)
print(profitability_metrics)

innovation_metrics = analyze_innovation(trades_df, historical_strategies)
print(innovation_metrics)

# Save blockchain
def save_blockchain(blockchain, filename='blockchain.pkl'):
    with open(filename, 'wb') as file:
        pickle.dump(blockchain, file)
    print(f"Blockchain saved to {filename}")

save_blockchain(blockchain)


INFO:root:Running backtest from 2019-01-01 00:00:00 to 2020-01-01 00:00:00.
INFO:root:Retrieving price data for universe


[<function create_historical_strategies.<locals>.strategy_buy at 0x000001DC1504DC60>, <function create_historical_strategies.<locals>.strategy_sell at 0x000001DC15E7ECA0>, <function create_historical_strategies.<locals>.strategy_random at 0x000001DC15E7CD60>, <function create_historical_strategies.<locals>.strategy_short_momentum at 0x000001DC15E7EB60>, <function create_historical_strategies.<locals>.strategy_long_momentum at 0x000001DC15E7DD00>, <function create_historical_strategies.<locals>.strategy_mean_reversion at 0x000001DC15E7F420>, <function create_historical_strategies.<locals>.strategy_moving_average at 0x000001DC15E7D940>, <function create_historical_strategies.<locals>.strategy_above_ma at 0x000001DC15E7E520>, <function create_historical_strategies.<locals>.strategy_volatility_breakout at 0x000001DC15E7CB80>, <function create_historical_strategies.<locals>.strategy_rsi at 0x000001DC15E7E700>]


c:\Users\ASUS\anaconda3\Lib\site-packages\pandas\core\frame.py:11211: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
c:\Users\ASUS\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\Users\ASUS\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
INFO:root:Stop loss triggered for NVDA at 2019-01-03 00:00:00. Selling all shares.
INFO:root:Stop loss triggered for NFLX at 2019-01-03 00:00:00. Selling all shares.
INFO:root:Stop loss triggered for CSCO at 2019-01-03 00:00:00. Selling all shares.
INFO:root:-----------------------------------
INFO:root:Rebalancing portfolio at 2019-01-31 00:00:00
INFO:root:-----------------------------------
INFO:root:Rebalancing portfolio at 2019-02-28 00:00:00
INFO:root:Stop loss triggered for TSLA at 2019-03-06 00:00:00. Selling a

Net Profit: 1143106718.414781
Sharpe Ratio: 0.491270304765847
{'Net Profit': 1143106718.414781, 'Sharpe Ratio': 0.491270304765847}
Error applying historical strategy: 'Close'
Error applying historical strategy: 'Close'
Error applying historical strategy: 'Close'
Error applying historical strategy: 'Close'
Error applying historical strategy: 'Close'
Error applying historical strategy: 'Close'
Error applying historical strategy: 'Close'
Innovation Score: 1
{'Innovation Score': 1}
Blockchain saved to blockchain.pkl


In [4]:
from pybacktestchain.data_module import FirstTwoMoments
from pybacktestchain.broker import *
from datetime import datetime
from Python-203.src.python203.backtestrating import BacktestRating

# Custom backtest parameters
initial_cash = 10000
start_date = datetime(2019, 1, 1)
end_date = datetime(2020, 1, 1)

# Initialize the backtest
backtest = CustomBacktest(
    initial_date=start_date,
    final_date=end_date,
    information_class=FirstTwoMoments,
    risk_model=StopLoss,
    initial_cash=initial_cash,
    name_blockchain='custom_backtest',
    verbose=False
)

# Initialize BacktestRating with the custom backtest
rating = BacktestRating(backtest)

# Run the backtest
rating.run_backtest()

# Calculate and print metrics
metrics = rating.calculate_metrics()
print("Backtest Metrics:")
for key, value in metrics.items():
    print(f"{key}: {value}")

SyntaxError: invalid decimal literal (2314413826.py, line 4)

In [9]:
from python203.testing import BacktestRating 